<div class="alert alert-warning">
    
<b>Disclaimer:</b> The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function
- setting and run the fit   
- extracting and displaying information about the results

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div> 

# Example: Composition of two Lorentzians convoluted with a resolution function and fitted with lmfit

## Table of Contents

- [Introduction](#Introduction)
- [Importing the required libraries](#Importing-the-required-libraries)
- [Importing the reference data](#Importing-the-reference-data)
- [Setting and fitting](#Setting-and-fitting)  
- [Plotting the results](#Plotting-the-results)  

[Top](#Table-of-Contents)

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use one of the models of 
the <a href="https://github.com/QENSlibrary/QENSmodels">QENSlibrary</a>, <b>Lorentzian</b>, to perform some fits.
<a href="https://lmfit.github.io/lmfit-py/">lmfit</a> is used for fitting.
</div>

The following example uses the data from IRIS:
- workspace_index=0, file: `irs26176_graphite002_red.nxs` 
- related instrument resolution data `irs26173_graphite002_res.nxs` 

The ISIS sample datasets can be downloaded from [Mantid's website](http://download.mantidproject.org/).
The data used for this example are in the sample datafile: `data_2lorentzians.dat` and the instrument resolution datafile `irf_iris.dat`, respectively.

This example is based on a [Mantid "Fitting QENS Peaks" tutorial](https://www.mantidproject.org/Fitting_QENS_Peaks).

The implementation with `lmfit` is based on https://lmfit.github.io/lmfit-py/model.html

This example requires an additional Python module `scipy.interpolate` to interpolate the tabulated data of the instrument resolution

### Physical units
For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "meV", 
                       'q': "1/Angstrom", 
                       'hwhm': "meV", 
                       'scale': "unit_of_signal.meV",
                       'center': "meV"}

[Top](#Table-of-Contents)

## Importing the required librairies

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np

import matplotlib.pyplot as plt

# for interactivity (plots, buttons...)
from pandas import DataFrame
import panel.widgets as pnw

path_to_data = './data/'

%matplotlib notebook

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = pnw.Button(name='Yes', button_type='success')
    buttonN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install the QENSmodels' library?", panel.Row(buttonY, buttonN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    if buttonY.clicks>0:
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels
    elif buttonN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):
    lmfitY = pnw.Button(name='Yes', button_type='success')
    lmfitN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install lmfit?", panel.Row(lmfitY, lmfitN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("lmfit"):
    if lmfitY.clicks>0:
         !{sys.executable} -m pip install lmfit
    elif lmfitN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
from lmfit import Model, CompositeModel
from scipy.interpolate import interp1d

# import model from QENS library
import QENSmodels

[Top](#Table-of-Contents)

## Importing the reference data

In [ ]:
# Create model - 2 lorentzians convoluted with instrument resolution - 6 parameters
# Step 1 load instrument resolution data
irf_iris = np.loadtxt(path_to_data + 'irf_iris.dat')
x_irf = irf_iris[:, 0]
y_irf = irf_iris[:, 1]

In [ ]:
# Step 2: create function for instrument resolution data (cubic interpolation between tabulated data points)
f = interp1d(x_irf, y_irf, kind='cubic', bounds_error=False, fill_value='extrapolate')

def irf_gate(x):
    """ Function defined from the interpolation of instrument resolution data 
    Used to define fitting model and plot """ 
    return f(x)

# plot tabulated data and interpolated data
xx = np.linspace(-.25, .25, 500)

fig0, ax0 = plt.subplots()
ax0.plot(x_irf, y_irf, 'b.', label='tabulated data')
ax0.plot(xx, irf_gate(xx), 'g--', label='extrapolated data')
ax0.legend()
ax0.set(xlabel='Energy transfer (meV)', title='Instrument resolution: plot tabulated data and interpolated data')
ax0.grid();

[Top](#Table-of-Contents)

## Setting and fitting

In [ ]:
# Step 3: create "double lorentzian" profile
def model_2lorentzians(x, scale1, center1, hwhm1, scale2, center2, hwhm2):
    return QENSmodels.lorentzian(x, scale1, center1, hwhm1) + QENSmodels.lorentzian(x, scale2, center2, hwhm2)

In [ ]:
# Step 4: create convolution function 
# code from https://lmfit.github.io/lmfit-py/model.html

def convolve(arr, kernel):
    # simple convolution of two arrays
    npts = min(len(arr), len(kernel))
    pad  = np.ones(npts)
    tmp  = np.concatenate((pad*arr[0], arr, pad*arr[-1]))
    
    out  = np.convolve(tmp, kernel, mode='valid')
    noff = int((len(out) - npts)/2)
    return out[noff:noff+npts]

In [ ]:
# Create model for the fit
gmodel = CompositeModel(Model(irf_gate), Model(model_2lorentzians), convolve) 

print('Names of parameters:', gmodel.param_names )
print('Independent variable(s):', gmodel.independent_vars )

In [ ]:
# Load reference data - extract x and y values
two_lorentzians_iris = np.loadtxt(path_to_data + 'data_2lorentzians.dat')
xx = two_lorentzians_iris[:, 0]
yy = two_lorentzians_iris[:, 1]

In [ ]:
# Fit
result = gmodel.fit(yy, x=xx, scale1=1., center1=0., hwhm1=0.25, scale2=1., center2=1., hwhm2=0.25)

fig1, ax1 = plt.subplots()
ax1.plot(xx, yy, '+', label='experimental data')
ax1.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
ax1.legend()
ax1.set(xlabel='Energy transfer (meV)', title='Plot before fitting: experimental data and mode with initial guesses')
ax1.grid();

[Top](#Table-of-Contents)

## Plotting results

In [ ]:
# display result
print('Result of fit:\n', result.fit_report())

# plot selected results: experimental data and best fit
fig2, ax2 = plt.subplots()
ax2.plot(xx, yy, '+', label='experimental data')
ax2.plot(xx, result.best_fit, 'r-', label='best fit')
ax2.grid()
ax2.set(xlabel='Energy transfer (meV)', title='Plot selected fitting results: experimental data and best fit')
ax2.legend();

In [ ]:
# other option to plot result using lmfit's features
result.plot()

In [ ]:
for item in result.params.keys():
    print(item[:-1], result.params[item].value, '+/-', result.params[item].stderr, dict_physical_units[item[:-1]])